In [1]:
import pandas as pd
import numpy as np
from scipy.special import comb

In [2]:
df = pd.read_csv('2020_ten_bent_coins.csv').transpose()

In [3]:
# O being tail and 1 as head
# counting number of heads and tails
np.random.seed(0)
heads = df.sum().to_numpy() #numpy array
tails = 100 - heads
selected_coin = np.random.randint(0,10,size=(500,)) #creating an array of 500 values with each one having value ranging from 1 to 10
_, count_selected_coin = np.unique(selected_coin,return_counts = True) # count of which coin has been selected how many times
MLE_vector = np.zeros(10) #maximum likelihood estimation


In [4]:
selected_coin

array([5, 0, 3, 3, 7, 9, 3, 5, 2, 4, 7, 6, 8, 8, 1, 6, 7, 7, 8, 1, 5, 9,
       8, 9, 4, 3, 0, 3, 5, 0, 2, 3, 8, 1, 3, 3, 3, 7, 0, 1, 9, 9, 0, 4,
       7, 3, 2, 7, 2, 0, 0, 4, 5, 5, 6, 8, 4, 1, 4, 9, 8, 1, 1, 7, 9, 9,
       3, 6, 7, 2, 0, 3, 5, 9, 4, 4, 6, 4, 4, 3, 4, 4, 8, 4, 3, 7, 5, 5,
       0, 1, 5, 9, 3, 0, 5, 0, 1, 2, 4, 2, 0, 3, 2, 0, 7, 5, 9, 0, 2, 7,
       2, 9, 2, 3, 3, 2, 3, 4, 1, 2, 9, 1, 4, 6, 8, 2, 3, 0, 0, 6, 0, 6,
       3, 3, 8, 8, 8, 2, 3, 2, 0, 8, 8, 3, 8, 2, 8, 4, 3, 0, 4, 3, 6, 9,
       8, 0, 8, 5, 9, 0, 9, 6, 5, 3, 1, 8, 0, 4, 9, 6, 5, 7, 8, 8, 9, 2,
       8, 6, 6, 9, 1, 6, 8, 8, 3, 2, 3, 6, 3, 6, 5, 7, 0, 8, 4, 6, 5, 8,
       2, 3, 9, 7, 5, 3, 4, 5, 3, 3, 7, 9, 9, 9, 7, 3, 2, 3, 9, 7, 7, 5,
       1, 2, 2, 8, 1, 5, 8, 4, 0, 2, 5, 5, 0, 8, 1, 1, 0, 3, 8, 8, 4, 4,
       0, 9, 3, 7, 3, 2, 1, 1, 2, 1, 4, 2, 5, 5, 5, 2, 5, 7, 7, 6, 1, 6,
       7, 2, 3, 1, 9, 5, 9, 9, 2, 0, 9, 1, 9, 0, 6, 0, 4, 8, 4, 3, 3, 8,
       8, 7, 0, 3, 8, 7, 7, 1, 8, 4, 7, 0, 4, 9, 0,

In [5]:
MLE_vector.shape

(10,)

In [6]:
for i,j in zip(heads, selected_coin):
  MLE_vector[j] += i

In [7]:
#The MLE vector is then divided by the product of the count of the selected coin and the total number of tosses (100) to obtain the MLE estimates of the unknown bias values.
MLE_vector = MLE_vector/(count_selected_coin*100)


In [8]:
#A function compute_likelihood is defined to calculate the likelihood of a given observation (number of heads) given the number of tosses and the estimated bias value.
def compute_likelihood(obs, n, pheads):

    likelihood = comb(n, obs, exact=True)*(pheads**obs)*(1.0-pheads)**(n-obs)

    return likelihood

In [9]:
np.random.seed(0)
p_heads = np.zeros((100,10))
p_heads[0]=np.random.random((1,10))
print(p_heads[0])
#The loop continues until the improvement in the MLE estimates between two consecutive iterations is less than a threshold eps, which is set to 0.01.
eps = 0.01
improvement = float('inf') #positive infinity
epoch = 0
while improvement>eps:
  expectation = np.zeros((10,500,2))
  
  for i in range(500):
    e_head = heads[i]
    e_tail = tails[i]

    likelihood = np.zeros(10)

    for j in range(10):
      likelihood[j]= compute_likelihood(e_head,100,p_heads[epoch][j])
    
    weights = likelihood/np.sum(likelihood)
    for j in range(10):
      expectation[j][i] = weights[j]*np.array([e_head,e_tail])
  
  theta = np.zeros(10)
  for i in range(10):
    theta[i] = np.sum(expectation[i],axis =0)[0]/np.sum(expectation[i])
  
  p_heads[epoch+1] = theta
  print(f'Epoch ->{epoch}\n Theta ->{theta}')

  improvement = max(abs(p_heads[epoch+1]-p_heads[epoch]))
  epoch+=1

[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548  0.64589411
 0.43758721 0.891773   0.96366276 0.38344152]
Epoch ->0
 Theta ->[0.54030055 0.74700458 0.60774813 0.53612429 0.39390495 0.66432208
 0.42455597 0.87913902 0.9408577  0.16211897]
Epoch ->1
 Theta ->[0.5302535  0.75955475 0.61332103 0.5258201  0.35874312 0.67708556
 0.40729141 0.87384035 0.92352302 0.10546534]
Epoch ->2
 Theta ->[0.52018909 0.76576297 0.61839452 0.5155784  0.31426446 0.68593045
 0.39912291 0.87020696 0.9136364  0.08328092]
Epoch ->3
 Theta ->[0.51163503 0.76889438 0.62203863 0.50672204 0.27727676 0.69178149
 0.39765551 0.86830699 0.90930678 0.07144459]
Epoch ->4
 Theta ->[0.50591337 0.77056051 0.62378851 0.50050643 0.24532125 0.69564221
 0.39109125 0.86756872 0.90765306 0.0584879 ]
Epoch ->5
 Theta ->[0.50177514 0.77150992 0.62403936 0.49571276 0.22302672 0.69828232
 0.38031544 0.86742743 0.90706774 0.04969973]
Epoch ->6
 Theta ->[0.49827412 0.77206431 0.62329137 0.49144947 0.20702191 0.70011169
 0.368592

In [10]:
#The MLE estimates are stored in the theta variable, which is the final output of the code.
for i, j in enumerate(theta): # to get the index as well as value
    print(f"{i+1} : {j:.3f}")

1 : 0.477
2 : 0.772
3 : 0.609
4 : 0.460
5 : 0.136
6 : 0.702
7 : 0.309
8 : 0.869
9 : 0.907
10 : 0.015


In [11]:
theta

array([0.47744018, 0.77207913, 0.60879011, 0.45986275, 0.13608127,
       0.70164984, 0.30932917, 0.86903319, 0.90663702, 0.01504909])